In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 10
    self.test_batch = 1000
    self.global_epochs = 100
    self.local_epochs = 2
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 100
    self.worker_num = 20
    self.sample_num = 20
    self.cluster_num = 2
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
# Tuned value
lr = 0

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for i in range(len(dataset)):
      self.data.append(dataset[i][0][0])
      self.target.append(dataset[i][1][0])

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_shakespeare.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_shakespeare.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[28, 6, 70, 62, 57, 35, 26, 139, 22, 108, 8, 7, 23, 55, 59, 129, 50, 107, 56, 114]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class RNN1(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN1, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN2(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN2, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

class RNN3(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN3, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=3, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output
    
class RNN4(nn.Module):
    def __init__(self, embedding_dim=8, vocab_size=90, hidden_size=256):
        super(RNN4, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=4, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, input_seq):
        embeds = self.embeddings(input_seq)
        # Note that the order of mini-batch is random so there is no hidden relationship among batches.
        # So we do not input the previous batch's hidden state,
        # leaving the first hidden state zero `self.lstm(embeds, None)`.
        lstm_out, _ = self.lstm(embeds)
        # use the final hidden state as the next character prediction
        final_hidden_state = lstm_out[:, -1]
        output = self.fc(final_hidden_state)
        return output

In [15]:
class Server():
  def __init__(self):
    self.models = []
    for i in range(args.cluster_num):
      self.models.append(RNN2())

  def model_initialize(self,workers):
    sample_worker = self.sample_worker(workers)
    self.send_models(sample_worker)
    for i,worker in enumerate(sample_worker):
      worker.cluster = i%args.cluster_num
      _ = worker.local_train()
    self.aggregate_models(sample_worker)

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_models(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.models = copy.deepcopy(self.models)
      for i in range(args.cluster_num):
        worker.models[i] = worker.models[i].to(args.device)

  def aggregate_models(self,workers):
    new_params = []
    for i in range(args.cluster_num):   
      new_params.append(OrderedDict())
    total_num = [0]*args.cluster_num
    for worker in workers:
      total_num[worker.cluster] += worker.train_data_num
    count = [0]*args.cluster_num    
    for worker in workers:
      worker_state = worker.models[worker.cluster].state_dict()
      for key in worker_state.keys():
        if count[worker.cluster]==0:
          new_params[worker.cluster][key] = 1.0*worker_state[key]*worker.train_data_num/total_num[worker.cluster]
        else:
          new_params[worker.cluster][key] += 1.0*worker_state[key]*worker.train_data_num/total_num[worker.cluster]
      count[worker.cluster] += 1
      for i in range(args.cluster_num):
        worker.models[i] = worker.models[i].to('cpu')
      del worker.models

    for i in range(args.cluster_num):
      if total_num[i]!=0:
        self.models[i].load_state_dict(new_params[i])

In [16]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.models = None
    self.cluster = None
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    self.aggregation_weight = None

  def local_train(self):
    acc_train,loss_train = train(self.models[self.cluster],args.criterion,self.trainloader,args.local_epochs)
    acc_valid,loss_valid = test(self.models[self.cluster],args.criterion,self.valloader)
    return acc_train,loss_train,acc_valid,loss_valid

  def clustering(self,models):
    for i in range(args.cluster_num):
      if i==0:
        cluster = 0
        _,loss = test(models[i],args.criterion,self.trainloader)
      else:
        _,tmp = test(models[i],args.criterion,self.trainloader)
        if tmp<loss:
          cluster = i
    self.cluster = cluster
    

In [17]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)

In [18]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [19]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [20]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_models(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    worker.clustering(worker.models)
    acc_train_tmp,loss_train_tmp,acc_valid_tmp,loss_valid_tmp = worker.local_train()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_models(sample_worker)
  '''
  for i in range(args.cluster_num):
    server.models[i].to(args.device)
  for worker in workers:
    worker.clustering(server.models)
    acc_valid_tmp,loss_valid_tmp = test(server.models[worker.cluster],args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  for i in range(args.cluster_num):
    server.models[i].to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()

Epoch1  loss:4.201233466466268  accuracy:9.54439483502384
Epoch2  loss:3.3141179588105945  accuracy:15.101252036950212
Epoch3  loss:3.3123446202940414  accuracy:15.101252036950212
Epoch4  loss:3.306882296005884  accuracy:15.10125203695021
Epoch5  loss:3.299684410625034  accuracy:15.10125203695021
Epoch6  loss:3.287763931685024  accuracy:15.109212845278442
Epoch7  loss:3.2674460957447686  accuracy:15.446789409585273
Epoch8  loss:3.2369401610559887  accuracy:14.931556565141184
Epoch9  loss:3.2055156489213306  accuracy:15.851263543966734
Epoch10  loss:3.1522547483444217  accuracy:16.027887386081986
Epoch11  loss:3.1114813973506292  accuracy:17.212489941683756
Epoch12  loss:3.044049673610263  accuracy:18.421511728019393
Epoch13  loss:3.0044684294197297  accuracy:20.698671854723216
Epoch14  loss:2.9764884203672404  accuracy:19.866483974125824
Epoch15  loss:2.955218358172311  accuracy:20.77859805934078
Epoch16  loss:2.9104464319017196  accuracy:22.98955768536012
Epoch17  loss:2.8822253137826

In [22]:
acc_test = []
loss_test = []

for i in range(args.cluster_num):
  server.models[i].to(args.device)

nums = 0
for worker in workers:
  nums += worker.test_data_num

start = time.time()

for i,worker in enumerate(workers):
  worker.aggregation_weight = 1.0*worker.test_data_num/nums
  worker.clustering(server.models)
  acc_tmp,loss_tmp = test(server.models[worker.cluster],args.criterion,worker.testloader)
  acc_test.append(acc_tmp)
  loss_test.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

end = time.time()

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 accuracy:45.96620132255694  loss:1.9606729064668929
Worker2 accuracy:42.955326460481096  loss:2.163719415664673
Worker3 accuracy:36.36363636363637  loss:2.4885435104370117
Worker4 accuracy:37.878787878787875  loss:2.4763472080230713
Worker5 accuracy:56.25  loss:1.847248911857605
Worker6 accuracy:47.61904761904762  loss:2.7726168632507324
Worker7 accuracy:60.0  loss:1.3037159442901611
Worker8 accuracy:45.45454545454545  loss:2.046386241912842
Worker9 accuracy:66.66666666666667  loss:1.3058642148971558
Worker10 accuracy:41.935483870967744  loss:2.0918877124786377
Worker11 accuracy:38.3177570093458  loss:2.1396853923797607
Worker12 accuracy:50.0  loss:1.6683387756347656
Worker13 accuracy:44.17887432536623  loss:2.011270046234131
Worker14 accuracy:35.92814371257485  loss:2.2958807945251465
Worker15 accuracy:50.0  loss:1.7816612720489502
Worker16 accuracy:45.16839785961599  loss:1.9780920147895813
Worker17 accuracy:39.743589743589745  loss:2.7724156379699707
Worker18 accuracy:48.648

In [24]:
acc_tune_test = []
loss_tune_test = []
acc_tune_valid = []
loss_tune_valid = []

start = time.time()

for i,worker in enumerate(workers):
    worker.models = copy.deepcopy(server.models)
    worker.models[worker.cluster] = worker.models[worker.cluster].to(args.device)
    _,_,acc_tmp,loss_tmp = worker.local_train()
    acc_tune_valid.append(acc_tmp)
    loss_tune_valid.append(loss_tmp)
    print('Worker{} Valid accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    
    acc_tmp,loss_tmp = test(worker.models[worker.cluster],args.criterion,worker.testloader)
    acc_tune_test.append(acc_tmp)
    loss_tune_test.append(loss_tmp)
    print('Worker{} Test accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    for i in range(args.cluster_num):
        worker.models[i] = worker.models[i].to('cpu')
    del worker.models

end = time.time()

acc_valid_avg = sum(acc_tune_valid)/len(acc_tune_valid)
loss_valid_avg = sum(loss_tune_valid)/len(loss_tune_valid)
print('Validation(tune)  loss:{}  accuracy:{}'.format(loss_valid_avg,acc_valid_avg))
acc_test_avg = sum(acc_tune_test)/len(acc_tune_test)
loss_test_avg = sum(loss_tune_test)/len(loss_tune_test)
print('Test(tune)  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 Valid accuracy:44.629516227801595  loss:2.052854988310072
Worker1 Test accuracy:44.15870683321087  loss:2.0860771451677596
Worker2 Valid accuracy:43.83954154727794  loss:2.0413084030151367
Worker2 Test accuracy:41.23711340206186  loss:2.138051986694336
Worker3 Valid accuracy:45.283018867924525  loss:2.155996084213257
Worker3 Test accuracy:27.272727272727273  loss:2.636253595352173
Worker4 Valid accuracy:40.50632911392405  loss:2.357844591140747
Worker4 Test accuracy:37.878787878787875  loss:2.454663038253784
Worker5 Valid accuracy:39.473684210526315  loss:2.6450445652008057
Worker5 Test accuracy:50.0  loss:1.7835218906402588
Worker6 Valid accuracy:29.166666666666668  loss:2.2516353130340576
Worker6 Test accuracy:42.857142857142854  loss:2.7330124378204346
Worker7 Valid accuracy:20.0  loss:3.1428864002227783
Worker7 Test accuracy:60.0  loss:1.3584991693496704
Worker8 Valid accuracy:51.89003436426117  loss:1.8967803716659546
Worker8 Test accuracy:51.239669421487605  loss:1.878007

In [26]:
cluster = [0]*args.cluster_num
for worker in workers:
  cluster[worker.cluster] += 1

print(cluster)

[0, 20]
